In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [2]:
arc = pd.read_csv ("/datasets/final_steel/data_arc.csv", parse_dates=[1, 2])
bulk = pd.read_csv ("/datasets/final_steel/data_bulk.csv")
bulk_time = pd.read_csv ("/datasets/final_steel/data_bulk_time.csv", parse_dates=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
gas = pd.read_csv ("/datasets/final_steel/data_gas.csv")
temp = pd.read_csv ("/datasets/final_steel/data_temp.csv", parse_dates=[1])
wire = pd.read_csv ("/datasets/final_steel/data_wire.csv")
wire_time = pd.read_csv ("/datasets/final_steel/data_wire_time.csv", parse_dates=[1,2,3,4,5,6,7,8,9])

In [3]:
display(arc.dtypes),
print('='*100),
display(arc.describe())
print('='*100),
display(arc.head(10))

key                              int64
Начало нагрева дугой    datetime64[ns]
Конец нагрева дугой     datetime64[ns]
Активная мощность              float64
Реактивная мощность            float64
dtype: object

,key,Активная мощность,Реактивная мощность
count,14876.000000,14876.000000,14876.000000
mean,1615.220422,0.670441,0.452592
std,934.571502,0.408159,5.878702
min,1.000000,0.030002,-715.504924
25%,806.000000,0.395297,0.290991
50%,1617.000000,0.555517,0.415962
75%,2429.000000,0.857034,0.637371
max,3241.000000,3.731596,2.676388


,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397
5,2,2019-05-03 11:34:14,2019-05-03 11:36:31,0.438063,0.283043
6,2,2019-05-03 11:38:50,2019-05-03 11:44:28,1.296415,0.892914
7,2,2019-05-03 11:46:19,2019-05-03 11:48:25,0.490377,0.305281
8,2,2019-05-03 11:49:48,2019-05-03 11:53:18,0.827743,0.516874
9,3,2019-05-03 12:06:54,2019-05-03 12:11:34,1.062053,0.671494


#### в данной таблице указано время нагрева сплава электродами и их мощностные параметры.

In [4]:
display(bulk.describe())
print('='*100),
display(bulk.head(10))

,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
count,3129.000000,252.000000,22.000000,1298.000000,1014.000000,77.000000,576.000000,25.000000,1.0,19.000000,176.000000,177.000000,2450.000000,18.000000,2806.000000,2248.000000
mean,1624.383509,39.242063,253.045455,113.879045,104.394477,107.025974,118.925347,305.600000,49.0,76.315789,83.284091,76.819209,260.471020,181.111111,170.284747,160.513345
std,933.337642,18.277654,21.180578,75.483494,48.184126,81.790646,72.057776,191.022904,NaN,21.720581,26.060347,59.655365,120.649269,46.088009,65.868652,51.765319
min,1.000000,10.000000,228.000000,6.000000,12.000000,11.000000,17.000000,47.000000,49.0,63.000000,24.000000,8.000000,53.000000,151.000000,16.000000,1.000000
25%,816.000000,27.000000,242.000000,58.000000,72.000000,70.000000,69.750000,155.000000,49.0,66.000000,64.000000,25.000000,204.000000,153.250000,119.000000,105.000000
50%,1622.000000,31.000000,251.500000,97.500000,102.000000,86.000000,100.000000,298.000000,49.0,68.000000,86.500000,64.000000,208.000000,155.500000,151.000000,160.000000
75%,2431.000000,46.000000,257.750000,152.000000,133.000000,132.000000,157.000000,406.000000,49.0,70.500000,102.000000,106.000000,316.000000,203.500000,205.750000,205.000000
max,3241.000000,185.000000,325.000000,454.000000,281.000000,603.000000,503.000000,772.000000,49.0,147.000000,159.000000,313.000000,1849.000000,305.000000,636.000000,405.000000


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,NaN,NaN,NaN,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,150.0,154.0
1,2,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,149.0,154.0
2,3,NaN,NaN,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,152.0,153.0
3,4,NaN,NaN,NaN,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.0,NaN,153.0,154.0
4,5,NaN,NaN,NaN,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203.0,NaN,151.0,152.0
5,6,NaN,NaN,NaN,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204.0,NaN,201.0,154.0
6,7,NaN,NaN,NaN,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204.0,NaN,152.0,154.0
7,8,NaN,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410.0,NaN,252.0,153.0
8,9,NaN,NaN,NaN,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,99.0,203.0
9,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203.0,NaN,102.0,204.0


#### сыпучие материалы подаются большим разнообразием и дозировки имеют широкий диапазон. Так же видно, что некоторые материалы заметно реже добавляются, чем другие.

In [5]:
display(bulk_time.dtypes),
print('='*100),
display(bulk_time.describe()),
print('='*100),
display(bulk_time.head(10))

key                 int64
Bulk 1     datetime64[ns]
Bulk 2     datetime64[ns]
Bulk 3     datetime64[ns]
Bulk 4     datetime64[ns]
Bulk 5     datetime64[ns]
Bulk 6     datetime64[ns]
Bulk 7     datetime64[ns]
Bulk 8     datetime64[ns]
Bulk 9     datetime64[ns]
Bulk 10    datetime64[ns]
Bulk 11    datetime64[ns]
Bulk 12    datetime64[ns]
Bulk 13    datetime64[ns]
Bulk 14    datetime64[ns]
Bulk 15    datetime64[ns]
dtype: object

,key
count,3129.000000
mean,1624.383509
std,933.337642
min,1.000000
25%,816.000000
50%,1622.000000
75%,2431.000000
max,3241.000000


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,NaT,NaT,NaT,2019-05-03 11:21:30,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 11:03:52,NaT,2019-05-03 11:03:52,2019-05-03 11:03:52
1,2,NaT,NaT,NaT,2019-05-03 11:46:38,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 11:40:20,NaT,2019-05-03 11:40:20,2019-05-03 11:40:20
2,3,NaT,NaT,NaT,2019-05-03 12:31:06,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 12:09:40,NaT,2019-05-03 12:09:40,2019-05-03 12:09:40
3,4,NaT,NaT,NaT,2019-05-03 12:48:43,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 12:41:24,NaT,2019-05-03 12:41:24,2019-05-03 12:41:24
4,5,NaT,NaT,NaT,2019-05-03 13:18:50,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 13:12:56,NaT,2019-05-03 13:12:56,2019-05-03 13:12:56
5,6,NaT,NaT,NaT,2019-05-03 13:59:24,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 13:53:27,NaT,2019-05-03 13:53:27,2019-05-03 13:53:27
6,7,NaT,NaT,NaT,2019-05-03 14:29:14,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 14:22:19,NaT,2019-05-03 14:22:19,2019-05-03 14:22:19
7,8,NaT,NaT,NaT,2019-05-03 15:04:05,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 14:55:46,NaT,2019-05-03 14:55:46,2019-05-03 14:55:46
8,9,NaT,NaT,NaT,2019-05-03 15:47:34,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 15:41:00,NaT,2019-05-03 15:41:00,2019-05-03 15:41:00
9,10,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 16:18:52,NaT,2019-05-03 16:18:52,2019-05-03 16:18:52


#### в данной таблице указано время добавление сыпучих материалов 

In [6]:
display(gas.dtypes),
print('='*100),
display(gas.describe()),
print('='*100),
display(gas.head(10))

key        int64
Газ 1    float64
dtype: object

,key,Газ 1
count,3239.000000,3239.000000
mean,1621.861377,11.002062
std,935.386334,6.220327
min,1.000000,0.008399
25%,812.500000,7.043089
50%,1622.000000,9.836267
75%,2431.500000,13.769915
max,3241.000000,77.995040


,key,Газ 1
0,1,29.749986
1,2,12.555561
2,3,28.554793
3,4,18.841219
4,5,5.413692
5,6,11.273392
6,7,9.836267
7,8,15.171553
8,9,8.216623
9,10,6.529891


#### из данной таблицы видно, что все образцы проходят продувку газом

In [7]:
display(temp.dtypes),
print('='*100),
display(temp.describe()),
print('='*100),
display(temp.head(10))

key                      int64
Время замера    datetime64[ns]
Температура            float64
dtype: object

,key,Температура
count,15907.000000,13006.000000
mean,1607.880870,1591.840920
std,942.212073,21.375851
min,1.000000,1191.000000
25%,790.000000,1581.000000
50%,1618.000000,1591.000000
75%,2427.000000,1601.000000
max,3241.000000,1705.000000


,key,Время замера,Температура
0,1,2019-05-03 11:16:18,1571.0
1,1,2019-05-03 11:25:53,1604.0
2,1,2019-05-03 11:29:11,1618.0
3,1,2019-05-03 11:30:01,1601.0
4,1,2019-05-03 11:30:39,1613.0
5,2,2019-05-03 11:37:27,1581.0
6,2,2019-05-03 11:38:00,1577.0
7,2,2019-05-03 11:49:38,1589.0
8,2,2019-05-03 11:55:50,1604.0
9,2,2019-05-03 11:58:24,1608.0


#### результирующая температура на каждой итерации обработки образца. Эти данные будут являться таргетом для нашей модели.

In [8]:
display(wire.dtypes),
print('='*100),
display(wire.describe()),
print('='*100),
display(wire.head(10))

key         int64
Wire 1    float64
Wire 2    float64
Wire 3    float64
Wire 4    float64
Wire 5    float64
Wire 6    float64
Wire 7    float64
Wire 8    float64
Wire 9    float64
dtype: object

,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
count,3081.000000,3055.000000,1079.000000,63.000000,14.000000,1.000,73.000000,11.000000,19.000000,29.000000
mean,1623.426485,100.895853,50.577323,189.482681,57.442841,15.132,48.016974,10.039007,53.625193,34.155752
std,932.996726,42.012518,39.320216,99.513444,28.824667,NaN,33.919845,8.610584,16.881728,19.931616
min,1.000000,1.918800,0.030160,0.144144,24.148801,15.132,0.034320,0.234208,45.076721,4.622800
25%,823.000000,72.115684,20.193680,95.135044,40.807002,15.132,25.053600,6.762756,46.094879,22.058401
50%,1619.000000,100.158234,40.142956,235.194977,45.234282,15.132,42.076324,9.017009,46.279999,30.066399
75%,2434.000000,126.060484,70.227558,276.252014,76.124619,15.132,64.212723,11.886057,48.089603,43.862003
max,3241.000000,330.314424,282.780152,385.008668,113.231044,15.132,180.454575,32.847674,102.762401,90.053604


,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,60.059998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,96.052315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,91.160157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,89.063515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,89.238236,9.114560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,99.109917,9.114560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,98.061600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,155.282394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,113.393279,9.143681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,81.244796,12.376000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Из этой таблицы видно, что проволки, как и сыпучих материалов, добавляют разное количество и видов проволки тоже много

In [9]:
display(wire_time.dtypes),
print('='*100),
display(wire_time.describe()),
print('='*100),
display(wire_time.head(10))

key                int64
Wire 1    datetime64[ns]
Wire 2    datetime64[ns]
Wire 3    datetime64[ns]
Wire 4    datetime64[ns]
Wire 5    datetime64[ns]
Wire 6    datetime64[ns]
Wire 7    datetime64[ns]
Wire 8    datetime64[ns]
Wire 9    datetime64[ns]
dtype: object

,key
count,3081.000000
mean,1623.426485
std,932.996726
min,1.000000
25%,823.000000
50%,1619.000000
75%,2434.000000
max,3241.000000


,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,2019-05-03 11:11:41,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2,2019-05-03 11:46:10,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,3,2019-05-03 12:13:47,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,4,2019-05-03 12:48:05,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,5,2019-05-03 13:18:15,2019-05-03 13:32:06,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,6,2019-05-03 13:59:02,2019-05-03 13:59:54,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,7,2019-05-03 14:28:46,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,8,2019-05-03 15:03:42,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,9,2019-05-03 15:42:43,2019-05-03 15:58:12,NaT,NaT,NaT,NaT,NaT,NaT,NaT
9,10,2019-05-03 16:20:25,2019-05-03 16:21:02,NaT,NaT,NaT,NaT,NaT,NaT,NaT


#### таблица со временем добавления проволоки

In [10]:
names = [bulk, gas, temp, wire]
for k in names:
    miss = []
    for i in arc['key'].unique():
        if i not in k['key'].unique():
            miss.append(i)
    print('Разногласия по номерам партий с arc:', len(miss))
    print(miss)
    print('='*20)
print('Всего партий в выборке:', len(arc.groupby(by='key')['key'].count()))

Разногласия по номерам партий с arc: 86
[51, 52, 53, 54, 55, 56, 72, 80, 81, 110, 151, 188, 225, 269, 302, 330, 331, 332, 343, 350, 661, 830, 874, 931, 933, 934, 960, 961, 964, 966, 983, 984, 1062, 1105, 1221, 1268, 1334, 1402, 1517, 1518, 1535, 1566, 1623, 1656, 1783, 1818, 1911, 1959, 1974, 1979, 2009, 2010, 2043, 2056, 2195, 2196, 2197, 2198, 2216, 2217, 2231, 2310, 2390, 2408, 2434, 2460, 2468, 2471, 2595, 2599, 2600, 2608, 2625, 2628, 2738, 2739, 2816, 2821, 2863, 2884, 2891, 3018, 3026, 3047, 3182, 3216]
Разногласия по номерам партий с arc: 2
[193, 259]
Разногласия по номерам партий с arc: 0
[]
Разногласия по номерам партий с arc: 134
[51, 52, 53, 54, 55, 56, 81, 82, 83, 84, 85, 88, 109, 197, 209, 210, 211, 212, 269, 330, 331, 332, 375, 376, 377, 378, 711, 712, 713, 714, 715, 744, 748, 754, 755, 796, 797, 798, 799, 800, 841, 929, 930, 931, 932, 933, 934, 1102, 1103, 1104, 1105, 1106, 1107, 1184, 1379, 1380, 1525, 1526, 1527, 1528, 1564, 1565, 1566, 1646, 1743, 1744, 1745, 1746, 1

## План решения задачи:
- соединим таблицы в одну обобщенную, после уточнения вопросов  
- заполним пропуски нулями (так как в таблицах измеряется количество добавляемых материалов - 0 никак не повлияет на обучение модели), приведем данные в таблице соотвутствующему данным виду  
- уберем из обобщенной таблицы все столбцы с датами и временем (предполагаю, что эти столбцы могут послужить причиной утечки данных для модели и испортить ее обобщающую способность)    
- проведем нормализацию, т.к. значения от таблицы к таблице варьируются в широком диапазоне  
- в качестве моделей будем использовать хорошо зарекомендовавшие себя XGB, LGBM, CatBoost, сравним их с DummyRegressor для проверки адекватности прогнозов 

# Решение задачи

##### Длительность времени между первым и последним замером температуры.

In [11]:
temp_time = pd.DataFrame(temp.groupby(by='key', as_index=False)['Время замера'].diff())

In [12]:
temp_time = pd.DataFrame(temp_time.groupby(level=0)['Время замера'].sum())

In [13]:
temp_time

,Время замера
0,00:14:21
1,00:21:45
2,00:21:40
3,00:06:28
4,00:12:42
...,...
3211,00:21:39
3212,00:12:29
3213,00:52:28
3214,00:56:15


##### Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.

In [14]:
arc['Суммарное время нагрева электродами'] = arc['Конец нагрева дугой'] - arc['Начало нагрева дугой']

In [15]:
arc_time = pd.DataFrame(arc.groupby(by='key')['Суммарное время нагрева электродами'].sum())

In [16]:
arc_time

,Суммарное время нагрева электродами
key,
1,00:18:18
2,00:13:31
3,00:10:55
4,00:12:21
5,00:14:29
...,...
3237,00:15:09
3238,00:09:06
3239,00:20:16


##### Количество запусков нагрева электродами.

In [17]:
count_electrod = pd.DataFrame(arc.groupby(by='key')['Начало нагрева дугой'].count())

In [18]:
count_electrod.rename(columns={'Начало нагрева дугой':'Количество запусков нагрева электродами'}, inplace=True)

In [19]:
count_electrod

,Количество запусков нагрева электродами
key,
1,5
2,4
3,5
4,4
5,4
...,...
3237,5
3238,3
3239,8


##### Введем понятия полной мощности и работы (усредненные по партиям). 
   $$ 
Полная мощность = √ (Активная  мощность^2 + Реактивная  мощность^2) 
$$

$$ 
Работа = мощность * время
$$

In [20]:
mean_active = (pd.DataFrame(arc.groupby(by='key')['Активная мощность'].mean()))**2
mean_reactive = (pd.DataFrame(arc.groupby(by='key')['Реактивная мощность'].mean()))**2

In [21]:
mean = mean_active.merge(mean_reactive, on='key')

In [22]:
mean['total_power'] = np.sqrt(mean['Активная мощность']+mean['Реактивная мощность'])

In [23]:
mean

,Активная мощность,Реактивная мощность,total_power
key,,,
1,0.951853,0.405321,1.164978
2,0.582397,0.249528,0.912099
3,0.255203,0.102282,0.597900
4,0.643705,0.265302,0.953419
5,0.700223,0.317150,1.008649
...,...,...,...
3237,0.618572,0.320007,0.968803
3238,0.493051,0.314042,0.898384
3239,0.340291,0.151284,0.701124


##### Возьмем последнюю температуру партии

In [24]:
temp_last = pd.DataFrame(temp.groupby(by='key')['Температура'].apply(lambda x: x.iloc[-1]))

In [25]:
temp_last.rename(columns={'Температура':'таргет_Температура'}, 
             inplace=True)

In [26]:
temp_last

,таргет_Температура
key,
1,1613.0
2,1602.0
3,1599.0
4,1625.0
5,1602.0
...,...
3237,NaN
3238,NaN
3239,NaN


##### Добавим фичей первую температуру партии

In [27]:
temp_first = pd.DataFrame(temp.groupby(by='key')['Температура'].apply(lambda x: x.iloc[0]))

In [28]:
temp_first.rename(columns={'Температура':'начальная_Температура'}, 
             inplace=True)

In [29]:
temp_first

,начальная_Температура
key,
1,1571.0
2,1581.0
3,1596.0
4,1601.0
5,1576.0
...,...
3237,1569.0
3238,1584.0
3239,1598.0


#####  объединим все в один датафрейм

In [30]:
total = mean.merge(count_electrod, on='key')

In [31]:
total = total.merge(arc_time, on='key')

In [32]:
total = total.merge(gas, on='key')

In [33]:
total = total.merge(bulk, on='key')

In [34]:
total = total.merge(wire, on='key')

In [35]:
total = total.merge(temp_last, on='key')

In [36]:
total = total.merge(temp_first, on='key')

In [37]:
total['Длительность времени между первым и последним замером температуры'] = temp_time

In [38]:
total.set_index('key', inplace=True)

#####  Удалим строки, где у таргета пропущенны значения (693 это достаточно большое число, но все же допустимое)

In [39]:
total.dropna(subset=['таргет_Температура'],inplace=True)

<font color='brown'> *небольшая ремарка  
- в этом моменте датасет представляет собой сборную таблицу из небольших фрагментов, где индексы - это ключи (номера партии) и удаляя нулевые таргеты я удалил партии целиком.
 

In [40]:
total.fillna(0, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  """Entry point for launching an IPython kernel.


##### приведем колонки с периодами времени в секунды, чтобы у моделей при обучении не возникало проблем с форматами данных

In [41]:
total['Суммарное время нагрева электродами'] = total['Суммарное время нагрева электродами'] / np.timedelta64(1, 's')
total['Длительность времени между первым и последним замером температуры'] = total['Длительность времени между первым и последним замером температуры'] / np.timedelta64(1, 's')

##### перемножив полную мощность в ваттах на время в секундах мы получим работу в джоулях

In [42]:
total['work'] = total['Суммарное время нагрева электродами'] * total['total_power']

##### осмотрим данные на наличие выбросов

In [43]:
total.describe()

,Активная мощность,Реактивная мощность,total_power,Количество запусков нагрева электродами,Суммарное время нагрева электродами,Газ 1,Bulk 1,Bulk 2,Bulk 3,Bulk 4,...,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9,таргет_Температура,начальная_Температура,Длительность времени между первым и последним замером температуры,work
count,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,...,2329.000000,2329.0,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000,2329.000000
mean,0.486394,13.974288,0.912424,4.695578,807.647488,11.375600,3.427651,1.422499,47.348218,37.303564,...,0.306550,0.0,0.978820,0.006361,0.311311,0.317113,1593.365393,1586.718763,1856.057106,755.119097
std,0.289996,661.313601,3.692427,1.606629,340.838388,6.392041,12.412132,19.082119,75.310430,58.643238,...,4.757746,0.0,8.693299,0.205705,4.402707,3.456561,11.200915,28.290792,1320.628476,1652.997564
min,0.034680,0.016916,0.228471,1.000000,57.000000,0.008399,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1541.000000,1191.000000,0.000000,15.158096
25%,0.277129,0.154108,0.659778,4.000000,581.000000,7.282948,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1587.000000,1571.000000,1186.000000,399.865414
50%,0.415257,0.238060,0.814850,5.000000,778.000000,10.100950,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1593.000000,1587.000000,1594.000000,636.209689
75%,0.627366,0.351128,0.985456,6.000000,993.000000,14.216688,0.000000,0.000000,80.000000,77.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1598.000000,1603.000000,2259.000000,940.592976
max,2.020423,31915.075396,178.648507,16.000000,4189.000000,77.995040,185.000000,325.000000,454.000000,281.000000,...,113.231044,0.0,180.454575,8.490040,102.762401,62.025600,1653.000000,1660.000000,23537.000000,77354.803463


In [44]:
total[total['Реактивная мощность'] > 15]

,Активная мощность,Реактивная мощность,total_power,Количество запусков нагрева электродами,Суммарное время нагрева электродами,Газ 1,Bulk 1,Bulk 2,Bulk 3,Bulk 4,...,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9,таргет_Температура,начальная_Температура,Длительность времени между первым и последним замером температуры,work
key,,,,,,,,,,,,,,,,,,,,,
2116,0.213601,31915.075396,178.648507,4,433.0,11.582547,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1578.0,1579.0,1932.0,77354.803463


##### однозначно выброс, полная мощность превосходит в сотни раз порог 3 квартиля.

In [45]:
total[total['Суммарное время нагрева электродами'] > 3000]

,Активная мощность,Реактивная мощность,total_power,Количество запусков нагрева электродами,Суммарное время нагрева электродами,Газ 1,Bulk 1,Bulk 2,Bulk 3,Bulk 4,...,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9,таргет_Температура,начальная_Температура,Длительность времени между первым и последним замером температуры,work
key,,,,,,,,,,,,,,,,,,,,,
1689,1.360566,0.704256,1.436949,15,4189.0,33.597044,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1606.0,1583.0,1581.0,6019.378326


##### так же является выбросом, так как уходит далеко за пределы 3 квартиля

In [46]:
total[total['Газ 1'] > 60]

,Активная мощность,Реактивная мощность,total_power,Количество запусков нагрева электродами,Суммарное время нагрева электродами,Газ 1,Bulk 1,Bulk 2,Bulk 3,Bulk 4,...,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9,таргет_Температура,начальная_Температура,Длительность времени между первым и последним замером температуры,work
key,,,,,,,,,,,,,,,,,,,,,
280,0.749235,0.272097,1.010610,9,1807.0,60.935689,0.0,0.0,126.0,0.0,...,0.0,0.0,0.0,0.0,46.002319,0.0,1600.0,1532.0,1662.0,1826.171666
322,0.536418,0.380646,0.957635,13,2658.0,77.995040,185.0,0.0,101.0,58.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,1616.0,1619.0,1649.0,2545.393302


##### сложно сказать, является ли данное количество газа для продувки аномальным, ведь готовятся разные сплавы и необходимость веществ для каждого из них очень сильно разнится. Примем все значения сыпучих материалов/проволоки/газа за нормальные.

In [47]:
total[total['Длительность времени между первым и последним замером температуры'] > 15000]

,Активная мощность,Реактивная мощность,total_power,Количество запусков нагрева электродами,Суммарное время нагрева электродами,Газ 1,Bulk 1,Bulk 2,Bulk 3,Bulk 4,...,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9,таргет_Температура,начальная_Температура,Длительность времени между первым и последним замером температуры,work
key,,,,,,,,,,,,,,,,,,,,,
681,0.343131,0.174994,0.719808,7,947.0,11.622287,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1597.0,1594.0,16669.0,681.658625
1793,0.173948,0.110075,0.532938,3,333.0,5.682379,0.0,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1601.0,1635.0,23537.0,177.468343
2077,0.442513,0.269883,0.844036,4,701.0,7.953611,0.0,0.0,118.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1592.0,1578.0,18948.0,591.668951
2239,0.279537,0.134836,0.643718,4,614.0,1.378762,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1586.0,1579.0,20923.0,395.243099


##### тоже достаточно сложно определить, является ли данное значение выбросом - возможно это просто тугоплавкий металл. Но все же предлагаю удалить данное максимальное значение - потери от этого минимальны, но мы застрахуем себя от ошибки в определения выброса

In [48]:
total.drop(1689, inplace=True)

In [49]:
total.drop(2116, inplace=True)

#####  проведем нормализацию данных

In [50]:
columns = ['Активная мощность', 'Реактивная мощность', 'total_power', 'Количество запусков нагрева электродами',
       'Суммарное время нагрева электродами', 'Газ 1', 'Bulk 1', 'Bulk 2',
       'Bulk 3', 'Bulk 4', 'Bulk 5', 'Bulk 6', 'Bulk 7', 'Bulk 8', 'Bulk 9',
       'Bulk 10', 'Bulk 11', 'Bulk 12', 'Bulk 13', 'Bulk 14', 'Bulk 15',
       'Wire 1', 'Wire 2', 'Wire 3', 'Wire 4', 'Wire 5', 'Wire 6', 'Wire 7',
       'Wire 8', 'Wire 9', 'начальная_Температура',
       'Длительность времени между первым и последним замером температуры',
       'work']

In [51]:
scaler = StandardScaler()

In [52]:
total[columns] = scaler.fit_transform(total[columns])

In [53]:
total.rename(columns={'Активная мощность': 'active_power',
                      'Реактивная мощность': 'reactive_power',
                      'Количество запусков нагрева электродами':'count_heat',
             'Суммарное время нагрева электродами':'heat_time_sum',
             'Газ 1':'gas',
             'таргет_Температура':'target_temp',
             'начальная_Температура':'first_temp',
             'Длительность времени между первым и последним замером температуры':'period_temp_measure'}, 
             inplace=True)

##### разобьем выборку на тренировочную, валидационную и тестовую

In [54]:
features = total.drop('target_temp', axis = 1)

In [55]:
target = total['target_temp']

In [56]:
train_features, test_features, train_target, test_target = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.40, 
                                                                            random_state=42)

In [57]:
train_features, valid_features, train_target, valid_target = train_test_split(train_features, 
                                                                                train_target, 
                                                                                test_size=0.5, 
                                                                                random_state=42)

##### приступим к обучению моделей и их тестированию

##### 1. DummyRegressor

In [58]:
dummy = DummyRegressor ()

In [59]:
dummy.fit(train_features, train_target)

DummyRegressor(constant=None, quantile=None, strategy='mean')

In [60]:
predict_train = dummy.predict (train_features)
predict_valid = dummy.predict (valid_features)
predict_test = dummy.predict (test_features)

In [61]:
print("Предсказания на трейне:", round(mean_absolute_error(train_target, predict_train), 3))
print("Предсказания на валидационной выборке:", round(mean_absolute_error(valid_target, predict_valid), 3))
print("Предсказания на тесте:", round(mean_absolute_error(test_target, predict_test), 3))

Предсказания на трейне: 7.908
Предсказания на валидационной выборке: 8.072
Предсказания на тесте: 8.267


##### Как видно, наш бейслайн это 8 градусов.

##### 2. CatBoostRegressor

In [62]:
cat = CatBoostRegressor (random_state=42, eval_metric='MAE')

In [63]:
cat.fit(train_features, train_target, 
        eval_set=(valid_features, valid_target), 
        early_stopping_rounds=50,
        verbose=50,
        use_best_model=True
       )

0:	learn: 7.8440189	test: 8.0253574	best: 8.0253574 (0)	total: 50.8ms	remaining: 50.8s
50:	learn: 6.1518783	test: 6.9426694	best: 6.9426694 (50)	total: 906ms	remaining: 16.9s
100:	learn: 5.4625578	test: 6.6190249	best: 6.6190249 (100)	total: 1.74s	remaining: 15.5s
150:	learn: 5.0355019	test: 6.4738356	best: 6.4738356 (150)	total: 2.52s	remaining: 14.1s
200:	learn: 4.7483942	test: 6.4053065	best: 6.4053065 (200)	total: 3.37s	remaining: 13.4s
250:	learn: 4.5120113	test: 6.3600385	best: 6.3600385 (250)	total: 4.14s	remaining: 12.4s
300:	learn: 4.3007859	test: 6.3137724	best: 6.3137724 (300)	total: 5.01s	remaining: 11.6s
350:	learn: 4.0628893	test: 6.2718167	best: 6.2710658 (349)	total: 5.79s	remaining: 10.7s
400:	learn: 3.8070920	test: 6.2603584	best: 6.2593820 (393)	total: 6.65s	remaining: 9.94s
450:	learn: 3.5824543	test: 6.2518015	best: 6.2456854 (433)	total: 7.51s	remaining: 9.14s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 6.245685395
bestIteration = 433

Shrink

In [64]:
predict_train = cat.predict (train_features)
predict_valid = cat.predict (valid_features)
predict_test = cat.predict (test_features)

In [65]:
print("Предсказания на трейне:", round(mean_absolute_error(train_target, predict_train), 3))
print("Предсказания на валидационной выборке:", round(mean_absolute_error(valid_target, predict_valid), 3))
print("Предсказания на тесте:", round(mean_absolute_error(test_target, predict_test), 3))

Предсказания на трейне: 3.658
Предсказания на валидационной выборке: 6.246
Предсказания на тесте: 6.198


#### CatBoost показал лучший скор. Именно эту модель я предлагаю использовать заказчику.

##### 3. XGBRegressor

In [66]:
xgb = XGBRegressor (random_state=42, n_jobs=-1)

In [67]:
xgb.fit(train_features, train_target, eval_metric="mae")

[12:59:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [68]:
predict_train = xgb.predict (train_features)
predict_valid = xgb.predict (valid_features)
predict_test = xgb.predict (test_features)

In [69]:
print("Предсказания на трейне:", round(mean_absolute_error(train_target, predict_train), 3))
print("Предсказания на валидационной выборке:", round(mean_absolute_error(valid_target, predict_valid), 3))
print("Предсказания на тесте:", round(mean_absolute_error(test_target, predict_test), 3))

Предсказания на трейне: 4.202
Предсказания на валидационной выборке: 6.38
Предсказания на тесте: 6.297


##### 4. LGBM Regressor

In [70]:
lgbm = LGBMRegressor (random_state=42)

In [71]:
lgbm.fit(train_features, train_target, eval_set=(valid_features, valid_target), verbose=0, eval_metric="mae")

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [72]:
predict_train = lgbm.predict (train_features)
predict_valid = lgbm.predict (valid_features)
predict_test = lgbm.predict (test_features)

In [73]:
print("Предсказания на трейне:", round(mean_absolute_error(train_target, predict_train), 3))
print("Предсказания на валидационной выборке:", round(mean_absolute_error(valid_target, predict_valid), 3))
print("Предсказания на тесте:", round(mean_absolute_error(test_target, predict_test), 3))

Предсказания на трейне: 1.521
Предсказания на валидационной выборке: 6.572
Предсказания на тесте: 6.319


#### GREEDSEARCHCV

##### LGBM

In [74]:
param_dist = {'iterations': (500, 750, 1000),
               'learning_rate': (0.005, 0.01, 0.1),
               'n_estimators': (150, 200, 220),
              'num_leaves': (25, 35, 40)
               }

In [75]:
lgbm_model = GridSearchCV(lgbm, 
                        param_dist, 
                        verbose = 1, 
                        n_jobs=-1 
                        )

In [78]:
lgbm_model.fit(train_features, train_target)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 26.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=42,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=-1,
             param_grid={'iterations': (500, 750, 1000),
                         'learning_rate': (0.005, 0.01, 0.1),
                         'n_estimators': (150, 200

In [79]:
lgbm_model.best_params_

{'iterations': 500,
 'learning_rate': 0.1,
 'n_estimators': 200,
 'num_leaves': 25}

In [81]:
lgbm = LGBMRegressor (random_state=42, iterations=500, learning_rate=0.1, n_estimators=200, num_leaves=25)

In [82]:
lgbm.fit(train_features, train_target, eval_set=(valid_features, valid_target), verbose=0, eval_metric="mae")

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', iterations=500, learning_rate=0.1,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=200, n_jobs=-1, num_leaves=25,
              objective=None, random_state=42, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [83]:
predict_train = lgbm.predict (train_features)
predict_valid = lgbm.predict (valid_features)
predict_test = lgbm.predict (test_features)

In [84]:
print("Предсказания на трейне:", round(mean_absolute_error(train_target, predict_train), 3))
print("Предсказания на валидационной выборке:", round(mean_absolute_error(valid_target, predict_valid), 3))
print("Предсказания на тесте:", round(mean_absolute_error(test_target, predict_test), 3))

Предсказания на трейне: 0.656
Предсказания на валидационной выборке: 6.645
Предсказания на тесте: 6.511


##### Прогонять все модели гридсерчем/рандомсерчем я не буду - слишком много времени это занимает, да и CatBoost в большинстве своем отлично работает на дефолтных параметрах, что и доказывает его скор. 

# Выводы по проделаной работе

#### Какие пункты плана были выполнены, а какие — нет?

Изначально план работы был достаточно общим и поэтому пунктов было немного. В ходе подготовки данных стало видно, что в том виде, в котором их выгрузил заказчик для обучения модели они мало пригодны - необходим фичеинженеринг. Поэтому большая часть шагов была добавлена уже после составления плана.

#### Какие трудности возникли?

Основной трудностью в данной работе оказалась разрозненность предоставленных данных. В таблице $arc$ измерения за каждый этап обработки партии, а в таблицах $bulk, wire, gas$ - по партиям, все таблицы имели огромное число пропусков и правильно слить их воедино оказалось достаточно сложной, но от этого и очень интересной, задачей. В этом мне сильно помогла документация $python$, а так же профильные форумы по программированию. Так же некоторую проблему вызвало недопонимание самой задачи (конечного таргета), где изначально я планировал предсказывать температуру на каждом этапе, но изучив данные понял, что это практически невозможно ввиду огромного количества пропусков на разных этапах обработки и такие предсказания будут делаться по принципу 'пальцем в небо'. Тогда я решил предсказывать среднюю температуру по партии, но это оказалось неверным решением, о чем мне поведал Николай уже на этапе ревьюинга кода (работу я сдал за день до консультации), а заодно и подсказал несколько идей для фичеинжинеринга, за что ему огромное спасибо. После этого как таковых трудностей уже не возникло, был четкий план и понимание что от меня требуется.

#### Какие шаги стали ключевыми в решении задачи?

ключевыми шагами, в моем понимании, стали:  
- фичеинженеринг
- правильное слияние данных в один датафрейм
- предобработка (заполнение пропусков, удаление партий с пропущеным таргетом, анализ на выбросы и их удаление, нормализация и тд.)  

#### Какая итоговая модель и какое у неё качество?

Итоговой моделью выбран $CatBoostRegressor$ с дефолтными параметрами со скором $MAE = 6.198$. При попытках тюнить модель, к сожалению, скор только ухудшался. Другие же две модели, даже с затюнеными параметрами, так же не смогли достигнуть порога дефолтного кэтбуста, а посему в работе они так же были оставлены 'по умолчанию', но для примера $LightGBM$, как самая быстрая в обучении модель, был прогнан на $GreedSearchCV$ для демонстрации умения подбора параметров в полуавтоматическом режиме.